# Tiling

## Libraries

Install `detectree2` from the GitHub repository. It will download the necessary dependencies (*i.e.,* `detectron2`, `pytorch`).

In [ ]:
!pip install git+https://github.com/PatBall1/detectree2.git

In [ ]:
import os
import shutil
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

from detectron2.engine import DefaultPredictor
from detectree2.preprocessing.tiling import tile_data_train, to_traintest_folders, tile_data
from detectree2.models.predict import predict_on_data
from detectree2.models.train import MyTrainer, setup_cfg
from detectree2.models.outputs import project_to_geojson, stitch_crowns, clean_crowns
import rasterio
import geopandas as gpd
import numpy as np


## Tiling

The following code performs the tiling on the 25-cm resolution composite from Cambridge, without the need of crown data.

In [ ]:
# Set tiling parameters
buffer = 20
tile_width = 160
tile_height = 160
threshold = 0
appends = str(tile_width) + "_" + str(buffer) + "_" + str(threshold)


## Cambridge
site_path = Path("drive/Shareddrives/detectree2_Cambridge/data/Cambridge")
img_path = site_path / 'CityCentre_2017_32630.tif'
out_dir = site_path / f'tiles_0.25m_{appends}'

# Read in the tiff file
data = rasterio.open(img_path)

# Tile RGB imagery
tile_data(data, out_dir, buffer, tile_width, tile_height)

There are 216 after tiling the original 0.25m-resolution RGB image. From these, 23 will be used for training and 9 for testing

In [ ]:
# Select a random sample for pretrained model evaluation
tile_names = list(out_dir.glob("*.tif"))
n_tiles = len(tile_names)
samp_size = 32
np.random.seed(0)
tile_ind = np.random.randint(high=n_tiles, low=0, size=samp_size)
tile_samp = [tile_names[i] for i in tile_ind]
png_samp = [Path(str(name)[:-3] + 'png') for name in tile_samp]

# Copy tiles to new folder
tiles_samp_path = site_path / f'tiles_0.25m_{appends}_samp'
tiles_samp_path.mkdir(exist_ok=True)

for i,_ in enumerate(tile_samp):
  shutil.copy(tile_samp[i], tiles_samp_path)
  shutil.copy(png_samp[i], tiles_samp_path)

Then, the training and testing datasets need to be split into several folders:
- Small training dataset
- Large training dataset
- Testing dataset

In [ ]:
train_small_dir = site_path / f'0.25m_160_20_0_train_small'
train_small_dir.mkdir(exist_ok=True)
train_large_dir = site_path / f'0.25m_160_20_0_train_large'
train_large_dir.mkdir(exist_ok=True)
test_dir = site_path / f'0.25m_160_20_0_test'
test_dir.mkdir(exist_ok=True)

rand_tiles = np.random.choice(samp_size, samp_size, replace=False)
tile_names = list(tiles_samp_path.glob("*.tif"))
tile_samp = [tile_names[i] for i in tile_ind]
png_samp = [Path(str(name)[:-3] + 'png') for name in tile_samp]

for i,_ in enumerate(tile_samp):
    
    if i < 12:
        shutil.copy(tile_samp[i], train_small_dir / 'rgb')
        shutil.copy(png_samp[i], train_small_dir / 'rgb')
    if i < 24:
        shutil.copy(tile_samp[i], train_large_dir / 'rgb')
        shutil.copy(png_samp[i], train_large_dir / 'rgb')
    else:
        shutil.copy(tile_samp[i], test_dir / 'rgb')
        shutil.copy(png_samp[i], test_dir / 'rgb')